In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

     |████████████████████████████████| 1.6MB 6.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 785 kB 6.8 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 56.0.0
    Uninstalling setuptools-56.0.0:
      Successfully uninstalled setuptools-56.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 46.9 MB 59 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 250 kB 5.1 MB/s 
     |████████████████████████████████| 334 kB 9.8 MB/s 
     |████████████████████████████████| 48 kB 3.5 MB/s 
     |████████████████████████████████| 48 kB 4.0 MB/s 
     |████████████████████████████████| 98 kB 561 kB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 206 kB 55.2 MB/s 
     |████████████████████████████████| 696 kB 52.3 MB/s 
     |████████████████████████████████| 27.4 MB 135 kB/s 
     |████████████████████████████████| 950 kB 72.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 115 kB 49.8 MB/s 
     |██████████████████

In [ ]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_data = TabularDataset('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

Loaded data from: /content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/train.csv | Columns = 20 / 20 | Rows = 26457 -> 26457


,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
5979,5979,M,Y,Y,1,189000.0,Working,Incomplete higher,Married,House / apartment,-15287,-2501,1,0,1,0,NaN,3.0,-11.0,1.0
9347,9347,F,N,N,1,54000.0,Pensioner,Lower secondary,Married,House / apartment,-12129,365243,1,0,0,0,NaN,3.0,-8.0,0.0
13299,13299,M,N,N,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-10396,-778,1,0,0,0,Laborers,2.0,-5.0,2.0
20912,20912,F,N,Y,0,157500.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-23596,365243,1,0,0,0,NaN,1.0,-12.0,2.0
22675,22675,F,Y,Y,0,360000.0,State servant,Higher education,Separated,House / apartment,-19305,-1905,1,0,0,0,Core staff,1.0,-58.0,0.0


In [ ]:
label = 'credit'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    500.000000
mean       1.602000
std        0.657618
min        0.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        2.000000
Name: credit, dtype: float64


In [ ]:
train_data_noindex=train_data.drop(['index'],axis=1)

In [ ]:
save_path = '/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path,eval_metric='log_loss').fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 19
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == float, but few unique label-values observed and label-values can be converted to int).
	3 unique label values:  [1.0, 0.0, 2.0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12520.42 MB
	Train Data (Original)  Memory Usage: 0.3 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of 

In [ ]:
predictor.leaderboard(train_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,-0.301002,-0.794235,0.236600,0.112029,0.963543,0.236600,0.112029,0.963543,1,True,6
1,RandomForestEntr,-0.306546,-0.826339,0.238280,0.112779,1.072393,0.238280,0.112779,1.072393,1,True,7
2,ExtraTreesEntr,-0.312569,-0.812786,0.241560,0.212461,0.866171,0.241560,0.212461,0.866171,1,True,10
3,ExtraTreesGini,-0.314444,-0.817890,0.250747,0.111863,0.960809,0.250747,0.111863,0.960809,1,True,9
4,KNeighborsDist,-0.463333,-2.316663,0.108136,0.107980,0.008837,0.108136,0.107980,0.008837,1,True,2
5,XGBoost,-0.510123,-0.795088,0.075067,0.011555,0.597519,0.075067,0.011555,0.597519,1,True,11
6,WeightedEnsemble_L2,-0.554852,-0.756107,0.493527,0.286499,6.726375,0.009146,0.001359,1.358748,2,True,14
7,LightGBMLarge,-0.625904,-0.783674,0.023838,0.024911,1.095822,0.023838,0.024911,1.095822,1,True,13
8,NeuralNetMXNet,-0.666656,-0.816927,0.249527,0.175426,5.942437,0.249527,0.175426,5.942437,1,True,12
9,NeuralNetFastAI,-0.686296,-0.792074,0.053544,0.044519,1.357397,0.053544,0.044519,1.357397,1,True,3


In [ ]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.756107       0.286499  6.726375                0.001359           1.358748            2       True         14
1              CatBoost  -0.764638       0.011185  2.439846                0.011185           2.439846            1       True          8
2         LightGBMLarge  -0.783674       0.024911  1.095822                0.024911           1.095822            1       True         13
3              LightGBM  -0.785962       0.018683  0.380036                0.018683           0.380036            1       True          5
4       NeuralNetFastAI  -0.792074       0.044519  1.357397                0.044519           1.357397            1       True          3
5            LightGBMXT  -0.792296       0.018307  0.363702                0.018307           0.363702        

In [ ]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  multiclass
AutoGluon identified the following types of features:
('category', []) : 8 | ['gender', 'car', 'reality', 'income_type', 'edu_type', ...]
('float', [])    : 3 | ['income_total', 'family_size', 'begin_month']
('int', [])      : 7 | ['index', 'child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'work_phone', ...]


In [ ]:
test_data=TabularDataset('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/test.csv')

Loaded data from: /content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/test.csv | Columns = 19 / 19 | Rows = 10000 -> 10000


In [ ]:
submit = TabularDataset('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/sample_submission.csv')
submit

Loaded data from: /content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/sample_submission.csv | Columns = 4 / 4 | Rows = 10000 -> 10000


,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0
...,...,...,...,...
9995,36452,0,0,0
9996,36453,0,0,0
9997,36454,0,0,0
9998,36455,0,0,0


In [ ]:
abc

,0.0,1.0,2.0
0,0.128462,0.193991,0.677548
1,0.092844,0.198636,0.708520
2,0.093560,0.169376,0.737064
3,0.085227,0.150016,0.764757
4,0.096148,0.199512,0.704340
...,...,...,...
9995,0.095835,0.172042,0.732122
9996,0.117664,0.237308,0.645028
9997,0.085154,0.143330,0.771517
9998,0.084955,0.177691,0.737354


In [ ]:
abc=predictor.predict_proba(test_data, model='WeightedEnsemble_L2')
abc.columns = ["a", "b", "c"]
# ans=pd.DataFrame(submit)
# ans=ans.astype(float)
# ans
abc

,a,b,c
0,0.136026,0.287483,0.576491
1,0.084714,0.182551,0.732736
2,0.103673,0.173269,0.723057
3,0.115289,0.146479,0.738232
4,0.088865,0.183924,0.727212
...,...,...,...
9995,0.087933,0.208444,0.703623
9996,0.098951,0.285881,0.615168
9997,0.071903,0.191590,0.736507
9998,0.121407,0.171166,0.707427


In [ ]:
type(abc)

pandas.core.frame.DataFrame

In [ ]:
submit['0'] = abc['a']
submit['1'] = abc['b']
submit['2'] = abc['c']
submit

,index,0,1,2
0,26457,0.136026,0.287483,0.576491
1,26458,0.084714,0.182551,0.732736
2,26459,0.103673,0.173269,0.723057
3,26460,0.115289,0.146479,0.738232
4,26461,0.088865,0.183924,0.727212
...,...,...,...,...
9995,36452,0.087933,0.208444,0.703623
9996,36453,0.098951,0.285881,0.615168
9997,36454,0.071903,0.191590,0.736507
9998,36455,0.121407,0.171166,0.707427


In [ ]:
submit.to_csv('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/Autogluon11.csv',index=False)